# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 20 2022 8:30AM,239878,10,8157966,Eywa Pharma Inc.,Released
1,Jun 20 2022 8:30AM,239891,10,CTF0011274,"Citieffe, Inc.",Executing
2,Jun 20 2022 8:30AM,239892,10,CTF0011275,"Citieffe, Inc.",Released
3,Jun 17 2022 4:53PM,239890,16,SSF-8157995,Sartorius Stedim Filters Inc.,Released
4,Jun 17 2022 4:44PM,239889,12,HH6230.1,Hush Hush,Released
5,Jun 17 2022 4:42PM,239888,16,8156820,Sartorius Bioprocess Solutions,Released
6,Jun 17 2022 4:42PM,239888,16,8157403,Sartorius Bioprocess Solutions,Released
7,Jun 17 2022 4:42PM,239888,16,8157411,Sartorius Bioprocess Solutions,Released
8,Jun 17 2022 4:38PM,239887,16,6598863,"SD Head USA, LLC",Released
9,Jun 17 2022 4:23PM,239886,16,6598857,"SD Head USA, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,239888,Released,3
20,239889,Released,1
21,239890,Released,1
22,239891,Executing,1
23,239892,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239888,NaN,NaN,3.0
239889,NaN,NaN,1.0
239890,NaN,NaN,1.0
239891,NaN,1.0,NaN
239892,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239824,10.0,4.0,5.0
239830,0.0,0.0,1.0
239845,5.0,24.0,19.0
239859,0.0,1.0,0.0
239863,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239824,10,4,5
239830,0,0,1
239845,5,24,19
239859,0,1,0
239863,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239824,10,4,5
1,239830,0,0,1
2,239845,5,24,19
3,239859,0,1,0
4,239863,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239824,10,4,5
1,239830,,,1
2,239845,5,24,19
3,239859,,1,
4,239863,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 20 2022 8:30AM,239878,10,Eywa Pharma Inc.
1,Jun 20 2022 8:30AM,239891,10,"Citieffe, Inc."
2,Jun 20 2022 8:30AM,239892,10,"Citieffe, Inc."
3,Jun 17 2022 4:53PM,239890,16,Sartorius Stedim Filters Inc.
4,Jun 17 2022 4:44PM,239889,12,Hush Hush
5,Jun 17 2022 4:42PM,239888,16,Sartorius Bioprocess Solutions
8,Jun 17 2022 4:38PM,239887,16,"SD Head USA, LLC"
9,Jun 17 2022 4:23PM,239886,16,"SD Head USA, LLC"
10,Jun 17 2022 4:15PM,239882,19,"GCH Granules USA, Inc."
11,Jun 17 2022 3:13PM,239873,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 20 2022 8:30AM,239878,10,Eywa Pharma Inc.,,,1
1,Jun 20 2022 8:30AM,239891,10,"Citieffe, Inc.",,1,
2,Jun 20 2022 8:30AM,239892,10,"Citieffe, Inc.",,,1
3,Jun 17 2022 4:53PM,239890,16,Sartorius Stedim Filters Inc.,,,1
4,Jun 17 2022 4:44PM,239889,12,Hush Hush,,,1
5,Jun 17 2022 4:42PM,239888,16,Sartorius Bioprocess Solutions,,,3
6,Jun 17 2022 4:38PM,239887,16,"SD Head USA, LLC",,,1
7,Jun 17 2022 4:23PM,239886,16,"SD Head USA, LLC",,,1
8,Jun 17 2022 4:15PM,239882,19,"GCH Granules USA, Inc.",,,1
9,Jun 17 2022 3:13PM,239873,16,Sartorius Bioprocess Solutions,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 8:30AM,239878,10,Eywa Pharma Inc.,1,,
1,Jun 20 2022 8:30AM,239891,10,"Citieffe, Inc.",,1,
2,Jun 20 2022 8:30AM,239892,10,"Citieffe, Inc.",1,,
3,Jun 17 2022 4:53PM,239890,16,Sartorius Stedim Filters Inc.,1,,
4,Jun 17 2022 4:44PM,239889,12,Hush Hush,1,,
5,Jun 17 2022 4:42PM,239888,16,Sartorius Bioprocess Solutions,3,,
6,Jun 17 2022 4:38PM,239887,16,"SD Head USA, LLC",1,,
7,Jun 17 2022 4:23PM,239886,16,"SD Head USA, LLC",1,,
8,Jun 17 2022 4:15PM,239882,19,"GCH Granules USA, Inc.",1,,
9,Jun 17 2022 3:13PM,239873,16,Sartorius Bioprocess Solutions,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 8:30AM,239878,10,Eywa Pharma Inc.,1,,
1,Jun 20 2022 8:30AM,239891,10,"Citieffe, Inc.",,1,
2,Jun 20 2022 8:30AM,239892,10,"Citieffe, Inc.",1,,
3,Jun 17 2022 4:53PM,239890,16,Sartorius Stedim Filters Inc.,1,,
4,Jun 17 2022 4:44PM,239889,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 8:30AM,239878,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Jun 20 2022 8:30AM,239891,10,"Citieffe, Inc.",NaN,1.0,NaN
2,Jun 20 2022 8:30AM,239892,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jun 17 2022 4:53PM,239890,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
4,Jun 17 2022 4:44PM,239889,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 8:30AM,239878,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Jun 20 2022 8:30AM,239891,10,"Citieffe, Inc.",0.0,1.0,0.0
2,Jun 20 2022 8:30AM,239892,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jun 17 2022 4:53PM,239890,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
4,Jun 17 2022 4:44PM,239889,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1439255,4.0,2.0,4.0
12,239889,1.0,0.0,0.0
16,1199424,8.0,0.0,0.0
19,719617,3.0,0.0,0.0
20,959362,43.0,28.0,15.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1439255,4.0,2.0,4.0
1,12,239889,1.0,0.0,0.0
2,16,1199424,8.0,0.0,0.0
3,19,719617,3.0,0.0,0.0
4,20,959362,43.0,28.0,15.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,4.0,2.0,4.0
1,12,1.0,0.0,0.0
2,16,8.0,0.0,0.0
3,19,3.0,0.0,0.0
4,20,43.0,28.0,15.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,4.0
1,12,Released,1.0
2,16,Released,8.0
3,19,Released,3.0
4,20,Released,43.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Completed,4.0,0.0,0.0,0.0,15.0
Executing,2.0,0.0,0.0,0.0,28.0
Released,4.0,1.0,8.0,3.0,43.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Completed,4.0,0.0,0.0,0.0,15.0
1,Executing,2.0,0.0,0.0,0.0,28.0
2,Released,4.0,1.0,8.0,3.0,43.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Completed,4.0,0.0,0.0,0.0,15.0
1,Executing,2.0,0.0,0.0,0.0,28.0
2,Released,4.0,1.0,8.0,3.0,43.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()